In [ ]:
import requests
from io import BytesIO
from PIL import Image

# ComfyUI 服务地址
COMFYUI_URL = "http://home.chrissong.top:3818"

def queue_prompt(prompt):
    """发送工作流到 ComfyUI 队列"""
    response = requests.post(f"{COMFYUI_URL}/prompt", json={"prompt": prompt})
    return response.json()

def get_image(filename, subfolder, folder_type):
    """从 ComfyUI 获取生成的图片"""
    data = {
        "filename": filename,
        "subfolder": subfolder,
        "type": folder_type
    }
    response = requests.get(f"{COMFYUI_URL}/view", params=data)
    return Image.open(BytesIO(response.content))

# 示例：创建一个简单的文本到图像的工作流
def create_text_to_image_prompt(prompt_text, model_name="v1-5-pruned-emaonly-fp16.safetensors"):
    # 这里简化了工作流定义，实际工作流需要根据你的节点配置调整
    prompt = {
        "3": {
            "class_type": "KSampler",
            "inputs": {
                "cfg": 8,
                "denoise": 1,
                "latent_image": ["5", 0],
                "model": ["4", 0],
                "negative": ["7", 0],
                "positive": ["6", 0],
                "sampler_name": "euler",
                "scheduler": "normal",
                "seed": 12345,
                "steps": 20
            }
        },
        "4": {
            "class_type": "CheckpointLoaderSimple",
            "inputs": {
                "ckpt_name": model_name
            }
        },
        "5": {
            "class_type": "EmptyLatentImage",
            "inputs": {
                "batch_size": 1,
                "height": 1024,
                "width": 1024
            }
        },
        "6": {
            "class_type": "CLIPTextEncode",
            "inputs": {
                "clip": ["4", 1],
                "text": prompt_text
            }
        },
        "7": {
            "class_type": "CLIPTextEncode",
            "inputs": {
                "clip": ["4", 1],
                "text": "ugly, blurry, low quality"
            }
        },
        "8": {
            "class_type": "VAEDecode",
            "inputs": {
                "samples": ["3", 0],
                "vae": ["4", 2]
            }
        },
        "9": {
            "class_type": "SaveImage",
            "inputs": {
                "filename_prefix": "ComfyUI",
                "images": ["8", 0]
            }
        }
    }
    return prompt

# 使用示例
if __name__ == "__main__":
    # 创建工作流
    prompt = create_text_to_image_prompt("An endless desert, fiery sun, blue sky")
    
    # 发送到 ComfyUI 处理
    response = queue_prompt(prompt)
    prompt_id = response["prompt_id"]
    print(f"Prompt queued with ID: {prompt_id}")
    
    # 获取生成的图像（实际使用时需要轮询等待完成）
    # 这里简化了流程，实际需要根据输出节点的信息获取图片
    # image = get_image("ComfyUI_00001_.png", "", "output")
    # image.show()

Prompt queued with ID: 1243cd6f-5b34-4621-923e-f00b55a224c4
